In [ ]:

#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json" 
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json" 

#r "nuget: XPlot.Plotly.Interactive, 4.0.6"

#r "nuget: FSharp.Data"

open FSharp.Data

let wb = WorldBankData.GetDataContext()
let br = wb.Countries.``Brazil``

let getInterval indicator starting ending =
    seq {
        for year, value in indicator do
            yield year, value
    }
    |> Seq.filter(fun (year,_) -> year >= starting && year <= ending)
    |> Seq.toList
    |> List.unzip

open XPlot.Plotly

let getScatter title (year, value) =
    Scatter( name = title, x = year, y = value)

"Ready"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json Installed Packages FSharp.Data, 4.2.8 XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

Ready

In [ ]:
let getGdp starting ending title = 
    let gdp = br.Indicators.``GDP growth (annual %)``
    getInterval gdp starting ending |> getScatter title
let chart = 
    seq {
        yield getGdp 2003 2011 "Lula"
        yield getGdp 2011 2016 "Dilma"
        yield getGdp 2016 2018 "Temer"
        yield getGdp 2018 2022 "Bolsonaro"
    }
    |> Chart.Plot
    |> Chart.WithTitle "GDP growth (annual %)"


display (chart.Display("svg"))

Error: System.ArgumentException: No formatter is registered for MIME type svg.
   at Microsoft.DotNet.Interactive.Formatting.Formatter.InferPreferredFormatter(Type actualType, String mimeType) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\Formatter.cs:line 556
   at Microsoft.DotNet.Interactive.Formatting.Formatter.<>c__DisplayClass52_0.<GetPreferredFormatterFor>b__0(ValueTuple`2 tuple) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\Formatter.cs:line 508
   at System.Collections.Concurrent.ConcurrentDictionary`2.GetOrAdd(TKey key, Func`2 valueFactory)
   at Microsoft.DotNet.Interactive.Formatting.Formatter`1.FormatTo(T obj, FormatContext context, String mimeType) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\Formatter{T}.cs:line 58
   at Microsoft.DotNet.Interactive.Formatting.Formatter.FormatTo[T](T obj, FormatContext context, String mimeType) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\Formatter.cs:line 303
   at Micros

In [ ]:
let getInflation starting ending title = 
    let gdp = br.Indicators.``Inflation, consumer prices (annual %)``
    getInterval gdp starting ending |> getScatter title
seq {
    yield getInflation 2003 2011 "Lula"
    yield getInflation 2011 2016 "Dilma"
    yield getInflation 2016 2018 "Temer"
    yield getInflation 2018 2022 "Bolsonaro"
}
|> Chart.Plot
|> Chart.WithTitle "Inflação, preços ao consumidor"

<!DOCTYPE html>

In [ ]:
let getUnemployment starting ending title = 
    let gdp = br.Indicators.``Unemployment, total (% of total labor force) (modeled ILO estimate)``
    getInterval gdp starting ending |> getScatter title
seq {
    yield getUnemployment 2003 2011 "Lula"
    yield getUnemployment 2011 2016 "Dilma"
    yield getUnemployment 2016 2018 "Temer"
    yield getUnemployment 2018 2022 "Bolsonaro"
}
|> Chart.Plot
|> Chart.WithTitle "Desemprego"

<!DOCTYPE html>